#### REPLACE `order_date` COLUMN WITHIN DOUBLE QUOTES AS "DOUBLE QUOTES" WAS MISSING IN `order_date` COLUMN

In [0]:
%sql
SELECT
value,
regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"') AS fixed_value
FROM
gizmobox_dev.bronze.orders_vw

#### LOAD THE TRANSFORMED DATA INTO TEMP VIEW

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW orders_temp_vw AS
SELECT
value,
regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"') AS fixed_value
FROM
gizmobox_dev.bronze.orders_vw

#### DEFINE / EXTRACT JSON DATA TYPE DYNAMICALLY FOR ALL ATTRIBUTES

In [0]:
%sql
SELECT
schema_of_json(fixed_value) AS schema
FROM
orders_temp_vw

#### LOAD TRANSFORMED DATA INTO SILVER SCHEMA AS DELTA TABLE

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gizmobox_dev.silver.orders_json
AS
SELECT from_json(fixed_value, 'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') AS json_value
FROM
orders_temp_vw

#### VALIDATE THE JSON OBJECT HAS BEEN LOADED INTO `ORDERS_JSON` TABLE. THIS TABLE IS USED AS FURTHER TRANSFORMATION

In [0]:
%sql
SELECT * FROM gizmobox_dev.silver.orders_json

#### EXTRACT ALL THE JSON OBJECTS AS COLUMNS FROM ARRAY

In [0]:
%sql
SELECT
json_value.customer_id::INT AS customer_id,
json_value.order_id::INT AS order_id,
json_value.order_date::DATE AS order_date,
json_value.order_status::STRING AS order_status,
json_value.payment_method::STRING AS payment_method,
json_value.total_amount::INT AS total_amount,
json_value.transaction_timestamp::TIMESTAMP AS transaction_timestamp,
json_value.items
FROM
gizmobox_dev.silver.orders_json

#### REMOVE DUPLICATES FORM ITEMS ARRAY

In [0]:
%sql
SELECT
json_value.customer_id::INT AS customer_id,
json_value.order_id::INT AS order_id,
json_value.order_date::DATE AS order_date,
json_value.order_status::STRING AS order_status,
json_value.payment_method::STRING AS payment_method,
json_value.total_amount::INT AS total_amount,
json_value.transaction_timestamp::TIMESTAMP AS transaction_timestamp,
array_distinct(json_value.items)
FROM
gizmobox_dev.silver.orders_json

#### EXPLODE ARRAY TO GET ALL THE ELEMENTS FROM ARRAY

In [0]:
%sql
SELECT
json_value.customer_id::INT AS customer_id,
json_value.order_id::INT AS order_id,
json_value.order_date::DATE AS order_date,
json_value.order_status::STRING AS order_status,
json_value.payment_method::STRING AS payment_method,
json_value.total_amount::INT AS total_amount,
json_value.transaction_timestamp::TIMESTAMP AS transaction_timestamp,
explode(array_distinct(json_value.items)) AS item
FROM
gizmobox_dev.silver.orders_json

#### LOAD THE ITEM EXPLODE INTO TEMP VIEW

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW orders_item_explode_temp_vw
AS
SELECT
json_value.customer_id::INT AS customer_id,
json_value.order_id::INT AS order_id,
json_value.order_date::DATE AS order_date,
json_value.order_status::STRING AS order_status,
json_value.payment_method::STRING AS payment_method,
json_value.total_amount::INT AS total_amount,
json_value.transaction_timestamp::TIMESTAMP AS transaction_timestamp,
explode(array_distinct(json_value.items)) AS item
FROM
gizmobox_dev.silver.orders_json

#### EXTRACT ALL THE REQUIRED COLUMNS FROM TEMP VIEW

In [0]:
%sql
SELECT
customer_id,
order_id,
order_date,
order_status,
payment_method,
total_amount,
transaction_timestamp,
item.category,
item.item_id,
item.name,
item.price,
item.quantity,
item.details.brand,
item.details.color
FROM
orders_item_explode_temp_vw

#### LOAD TRANSFORMED DATA INTO SILVER SCHEMA AS DELTA TABLE

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gizmobox_dev.silver.orders
AS
SELECT
customer_id,
order_id,
order_date,
order_status,
payment_method,
total_amount,
transaction_timestamp,
item.category,
item.item_id,
item.name,
item.price,
item.quantity,
item.details.brand,
item.details.color
FROM
orders_item_explode_temp_vw

#### VALIDATE RECORDS HAVE BEEN LOADED INTO `GIZMOBOX_DEV.SILVER.ORDERS`

In [0]:
%sql
SELECT * FROM gizmobox_dev.silver.orders LIMIT 10

In [0]:
dbutils.notebook.exit('ORDERS TABLE HAS BEEN LOADED IN SILVER SCHEMA....')